# 02807 Computational Tools for Data Science
##  Book recomendation system 
### Introduction
The notebook first loads two dataframes, one containing data for each book, and one containing reviews of the books.

Then the data is cleaned.

Furthermore, different methods are used to recommend books.



<a id="0"></a> <br>
 # Table of Contents  
1. [Data](#data)    

In [ ]:
%env VARIABLE=comptools-env

### Load packages

In [ ]:
import pandas as pd
import numpy as np
import mmh3

<a id="data"></a> <br>
## Data


In [ ]:
books_data = pd.read_csv('data/books_data.csv') #, nrows=1000)
print(f"Books data has " + str(books_data.columns.size) + " columns with " + str(books_data.shape[0]) + " rows each.")
books_ratings = pd.read_csv('data/books_rating.csv') #, nrows=1000)
print(f"Books ratings has " + str(books_ratings.columns.size) + " columns with " + str(books_ratings.shape[0]) + " rows each.")

### Data Preprocessing
The following section cleans the data. 
This is done with the following steps

- We remove columns that we do not use and rename some of the columns.
- We change "&" to "and", remove special characters in the titles, and make titles lower case. This is done for both data sets.
- We remove dublicates of titles and descriptions.
- We remove duplicate reviews where same user reviews same book multiple times.
- We then remove reviews where title is NaN.
- Remove books with less than 10 reviews.
- Remove users with less than 20 reviews.
- Add column to book_data: average_rating and helpfullness of reviews to books_ratings

The data cleaning is now complete and we can go to the first model.

In [ ]:
# Titles
books_data_1 = books_data.drop(['image', 'previewLink', 'infoLink', 'publisher', 'ratingsCount'], axis=1, inplace=False)
books_ratings_1 = books_ratings.drop(['Id','Price', 'profileName', 'review/time'], axis=1, inplace=False)

books_data_1.rename(columns={'Title': 'title','publishedDate': 'published_date'}, inplace=True)
books_ratings_1.rename(columns={'Title': 'title','User_id':'user_id',  'review/score': 'score','review/helpfulness': 'helpfulness', 'review/text': 'text', 'review/summary': 'summary'}, inplace=True)
print(f"Books data now only has " + str(books_data_1.columns.size) + " columns - " + str(list(books_data_1.columns)))
print(f"Books ratings now only has " + str(books_ratings_1.columns.size) + " columns - " + str(list(books_ratings_1.columns))) 


In [ ]:
#Makes all titles lowercase; then removes any part of the title that includes a parenthesis, and replaces any ampersands with 'and'
books_data_1['title'] = books_data_1['title'].map(lambda s: s.lower() if type(s) == str else s)
books_data_1['title'] = books_data_1['title'].replace(r"\(.*\)","", regex=True)
books_data_1['title'] = books_data_1['title'].replace(r"\&","and", regex=True)

books_ratings_1['title'] = books_ratings_1['title'].map(lambda s: s.lower() if type(s) == str else s)
books_ratings_1['title'] = books_ratings_1['title'].replace(r"\(.*\)","", regex=True)
books_ratings_1['title'] = books_ratings_1['title'].replace(r"\&","and", regex=True)

#Removes any characters that aren't letters or spaces.
books_data_1['title'] = books_data_1['title'].replace(r"[^a-zA-Z0-9\s]+", "", regex=True)
books_ratings_1['title'] = books_ratings_1['title'].replace(r"[^a-zA-Z0-9\s]+", "", regex=True)

print("Both datasets' titles are now all lowercase and only include letters and spaces.")


In [ ]:
# keep only unique titles in books_data_1
#size of books_data_1
prev_size = len(books_data_1['title'])
print(f"Books data has " + str(len(books_data_1['title'])) + " titles.")
books_data_1 = books_data_1.drop_duplicates(subset=['title'], keep='first')
#books_data_1 = books_data_1.drop_duplicates(subset=['description'], keep='first')
print(f"Dropped " + str(prev_size - len(books_data_1['title'])) + " duplicate titles and descriptions. Now there are only " + str(len(books_data_1['title'])) + " rows left. ")

In [ ]:
#Drop duplicate reviews where same user reviews same book multiple times
print(f"Books ratings has " + str(len(books_ratings_1['title'])) + " rows.")
prevRatings = len(books_ratings_1['title'])
books_ratings_1.drop_duplicates(keep='first', inplace=True)
print(f"Dropped based on total duplicity. Books ratings now has " + str(len(books_ratings_1['title'])) + " rows. " + " -" + str(prevRatings-len(books_ratings_1['title'])) + " rows.")
prevRatings = len(books_ratings_1['title'])
books_ratings_1.drop_duplicates(subset=['user_id', 'title'], keep='first', inplace=True)
print(f"Dropped based on duplicate title. Books ratings now has " + str(len(books_ratings_1['title'])) + " rows. " + " -" + str(prevRatings-len(books_ratings_1['title'])) + " rows.")
prevRatings = len(books_ratings_1['title'])
books_ratings_1.drop_duplicates(subset=['user_id', 'text'], keep='first', inplace=True)
print(f"Dropped based on duplicate text. Books ratings now has " + str(len(books_ratings_1['title'])) + " rows. " + " -" + str(prevRatings-len(books_ratings_1['title'])) + " rows.")


In [ ]:
#Remove NAN titles
print(f"" + str(books_ratings_1.isnull().sum().sum()) + " total NAN values present in title")
books_ratings_1 = books_ratings_1.dropna(subset=['title'])
print(f"Now there are " + str(books_ratings_1.isnull().sum().sum()))

print(f"Now there are " + str(books_data_1.isnull().sum().sum()))

In [ ]:
#Collect only ratings for books that have received at least 10 ratings.
#Add a new column in books_data_1 that counts the number of ratings for each book with a score of at least 10

#Note that by removing duplicate reviews first, counting the number of reviews for each book afterward will be more accurate.
print(f"Books_ratings has " + str(len(books_ratings_1['title'])) + " rows.")
prev_rows = len(books_ratings_1['title'])

n_ratings = books_ratings_1.groupby('title').count()['score']>=10
famous_books = n_ratings[n_ratings==True].index
books_ratings_1 = books_ratings_1[books_ratings_1['title'].isin(famous_books)]
n_ratings_1 = books_ratings_1['title'].value_counts().rename("n_ratings")
books_data_1 = books_data_1.merge(n_ratings_1, on='title', how='inner')

print(f"Books_ratings now has " + str(len(books_ratings_1['title'])) + " rows. -" + str(prev_rows - len(books_ratings_1['title'])) + " rows. ")
print(f"Books_data now has an additional column. (n_ratings)")

#purge books where n_ratings < 10 (redundant operation)
prevBooks = len(books_data_1['title'])
books_data_1 = books_data_1[books_data_1['n_ratings']>=10]
print(f"Books data has " + str(books_data_1.columns.size) + " columns with " + str(books_data_1.shape[0]) + " rows each." + " -" + str(prevBooks-len(books_data_1['title'])) + " rows.")

In [ ]:
#Remove ratings from users who have rated less than 20 books.
prevRatings = len(books_ratings_1['title'])
x = books_ratings_1.groupby('user_id').count()['score'] >= 20 # since this is what amazon requires before a user can get recommendations
considerable_users = x[x].index
books_ratings_1 = books_ratings_1[books_ratings_1['user_id'].isin(considerable_users)]

print(f"Books ratings has " + str(books_ratings_1.columns.size) + " columns with " + str(books_ratings_1.shape[0]) + " rows each." + " -" + str(prevRatings-len(books_ratings_1['title'])) + " rows.")
print(f"" + str(len(books_ratings_1['user_id'].value_counts())) + " users remain who have rated at least 20 books.")

In [ ]:
# add column in books_data_1 that calculate  the average score of the book.
avg_rating = books_ratings_1.groupby('title')["score"].mean().rename("avg_rating")
books_data_1 = books_data_1.merge(avg_rating, on='title', how='inner')
print(f"Books data now has an additional column. (avg_rating)")

# function to get number of users who found the review helpful
def get_n_helpful(x):
    if isinstance(x, str) and '/' in x:
        num, denom = x.split('/')
        return int(denom)
    else:
        return x

# function to get percentage of users who found the review helpful
def replace_fraction(x):
    if isinstance(x, str) and '/' in x:
        num, denom = x.split('/')
        if denom == '0':
            return 0
        else:
            return int(num) / int(denom)
    else:
        return x

# add columns for number of users who found the review helpful and percentage of users who found the review helpful
books_ratings_1['helpfulness_count'] = books_ratings_1['helpfulness'].apply(get_n_helpful)
books_ratings_1['helpfulness_pct'] = books_ratings_1['helpfulness'].apply(replace_fraction)

## Recommendation Models

### Jaccard Similarity on Books Data


Function jaccard, that takes two titles and outputs the estimated jaccard similarity. \
Function max_jaccard takes a list of titles and compare the titles with each other. It then returns the two titles with the highest jaccard similarity.

In [ ]:
def jaccard(title1, title2):
    words1 = set(str(title1).lower().split())
    words2 = set(str(title2).lower().split())
    
    # Compute the intersection and union of the sets
    intersection = len(words1.intersection(words2))
    union = len(words1) + len(words2) - intersection
    
    # Calculate the Jaccard similarity
    similarity = intersection / union if union > 0 else 0.0
    
    return similarity

def max_jaccard(title_list):
    max_similarity = 0.0
    idx1 = 0
    idx2 = 0
    for i in range(len(title_list)):
        for j in range(i + 1, len(title_list)):
            similarity = jaccard(title_list[i], title_list[j])
            if similarity > max_similarity:
                idx1 = i
                idx2 = j
                max_similarity = similarity
    return max_similarity,idx1,idx2

"""
Example usage for comparing two titles
title1 = books_data_1['title'][0]
title2 = books_data_1['title'][1]
print(title1,title2)
similarity = jaccard(title1, title2)
print(f"Jaccard Similarity: {similarity}")
"""

# Example usage with a list of titles
n = 100
title_list = books_data_1['title'][0:n]
max_similarity,idx1,idx2 = max_jaccard(title_list)
print(f"Max Jaccard Similarity: {max_similarity}")
print(books_data_1['title'][idx1])
print(books_data_1['title'][idx2])


## Similar items using Jaccard similarity

Function for finding similar items yusing jaccard similarity.

In [ ]:
def similar(names, jaccard_threshold=0.6):
    # Create a dictionary to store the similar names
    similar_names = {}
    
    # Loop through each name in the list
    for i in range(len(names)):
        for j in range(i+1, len(names)):
            similarity_score = jaccard(names[i], names[j])
            if similarity_score >= jaccard_threshold:
                similar_names[(names[i], names[j])] = similarity_score
    return similar_names

# Example usage:    
names = books_data_1['title'][0:500]
similar_names = similar(names)
# Print titles in a way that is easier to read

for (desc1, desc2), score in similar_names.items():
    print(f"Similarity Score: {score}")
    print(f"Title 1: {desc1}")
    print(f"Title 2: {desc2}")
    print()


In [ ]:
# Example usage for finding similar descriptions
descritions = books_data_1['description'][0:400]
similar_descriptions = similar(descritions,jaccard_threshold=0.4)

# Print descriptions in a way that is easier to read
for (desc1, desc2), score in similar_descriptions.items():
    print(f"Similarity Score: {score}")
    print(f"Description 1: {desc1}")
    print(f"Description 2: {desc2}")
    print()


## Locality-Sensitive Hashing

Changing to keys for faster computations

In [ ]:


names = books_data_1['title'][0:2]
names = names.to_dict()
print(names)

for key, values in names.items():
    print(key)
    print(values)

In [ ]:
# Function to split words into k parts
def split_k(word,k):
    if word is np.nan:
        return []
    else:
        n = len(word)
    if k > n:
        return split_k(word,int(k/2))
    else:
        if k == 0:
            return word
        n1 = n//k
        n2 = n1 + n%k
        return [word[i:i+n1] for i in range(0, n, n1)]

In [ ]:
# Implement the LSH algorithm
b = 10

def lsh(names, jaccard_threshold,seed):
    lsh_dict = {}
    for key, name in names.to_dict().items():
        blocks = split_k(name,b)
        blocks_hash_values = []
        for aBlock in blocks:
            blocks_hash_values.append(mmh3.hash(aBlock, seed))
        lsh_dict[key] = blocks_hash_values
    list_keys = list(lsh_dict.keys())
    similar_items = {}
    for i in range(len(list_keys)-1):
        if i% 500 == 0:
            print(i)
        for j in range(i+1, len(list_keys)):
            common_values = np.intersect1d(lsh_dict[list_keys[i]], lsh_dict[list_keys[j]])
            if len(common_values) > 0:
                # we found a candidate
                similarity_score = jaccard(names[list_keys[i]], names[list_keys[j]])
                if similarity_score >= jaccard_threshold:
                    similar_items[(list_keys[i], list_keys[j])] = similarity_score
    return similar_items

n = 200


titles = books_data_1['title'][0:n]
found_similar_items_with_lsh = lsh(titles,jaccard_threshold=0.6,seed=42)
# Print the results
for (title1, title2), score in found_similar_items_with_lsh.items():
    print(f"Similarity Score: {score}")
    print(f"Title 1: {titles[title1]}")
    print(f"Title 2: {titles[title2]}")
    print()
    if score == 1:
        books_data_1 = books_data_1.drop(title2)
        
